# 🌿 SmartBloom Disease Detector — Fixed Colab Notebook
### Models: **YOLOv8L** & **YOLO11L**
### Includes: Colab crash fix, Google Drive auto-export, model comparison

In [ ]:
# 🔧 Fix Colab crash issue
import os
os.environ['COLAB_DISABLE_OBJ_REPR'] = 'True'
print('Colab rendering fix applied!')

In [ ]:
!pip install ultralytics==8.3.0
from ultralytics import YOLO
from google.colab import files
import zipfile, os

## 📂 Upload Dataset ZIP

In [ ]:
uploaded = files.upload()
zip_name = next(iter(uploaded))
with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall('/content')
print('Dataset extracted!')

## 📝 Set `data.yaml` Path

In [ ]:
DATA_YAML = '/content/plant_disease/data.yaml'  # change if needed
print('Using dataset:', DATA_YAML)

## 🚀 Train YOLOv8L & YOLO11L with Safe Output Handling

In [ ]:
MODELS = ['yolov8l.pt', 'yolo11l.pt']
results_paths = {}

for m in MODELS:
    print(f"\n=== Training {m} ===")
    model = YOLO(m)
    _ = model.train(data=DATA_YAML, epochs=100, imgsz=640, batch=16, device=0)
    best = model.ckpt_path.replace('last.pt','best.pt')
    results_paths[m] = best

print('Training finished!')

## 📤 Auto-Export Best Weights to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_dir = '/content/drive/MyDrive/SmartBloom/weights/'
os.makedirs(save_dir, exist_ok=True)

for m, path in results_paths.items():
    if os.path.exists(path):
        !cp "$path" "$save_dir"
        print(f"Saved {m} best.pt → {save_dir}")